🚨 Smart 911 Medical-Dispatch Agent – Design Document

1. Environment Definition
Perceivable Inputs

    Caller utterance transcript (text or ASR)
    Caller phone number → identity, home address, prior medical history (from EMS CRM)
    Real-time GPS coordinate or reverse-geocoded address
    Environmental sound cues (breathing noise, crowd, sirens) via audio stream
    Current system load (available ambulances, hospital diversion status)

2. Tool List & Interfaces

| Tool / API                          | Consumes                                                     | Returns                                        |
| ----------------------------------- | ------------------------------------------------------------ | ---------------------------------------------- |
| **Infermedica Symptom Checker**     | Parsed symptoms (ICD-10 codes, free-text), age, sex          | Ranked differential diagnoses + severity flags |
| **Ambulance Scheduling System**     | Location, urgency, special needs (e.g., bariatric stretcher) | ETA of nearest unit + unit ID                  |
| **Medical Triage LLM (fine-tuned)** | Full transcript, caller tone, Infermedica output             | Urgency score 0-100, risk keywords             |
| **GIS / Traffic Service**           | Origin & destination coordinates                             | Real-time travel time, alternate routes        |


3. State Schema (JSON)

In [ ]:
{
  "call_id": "uuid",
  "caller": {
    "phone": "+1-xxx-xxx-xxxx",
    "identity": { "first_name": "...", "dob": "...", "allergies": [...] },
    "location": { "lat": 0.0, "lon": 0.0, "address": "..." }
  },
  "symptoms": [
    { "raw": "chest pain", "icd": "R06.02", "severity": 8 }
  ],
  "urgency_score": 92,
  "actions_taken": [
    { "type": "dispatch", "unit_id": "A-12", "eta": 210 },
    { "type": "advice", "text": "Keep patient seated, chew aspirin if available" }
  ],
  "decision_log": [
    { "step": "triage_llm", "timestamp": "...", "output": {...} }
  ]
}

4. Decision-Making Flowchart (Pseudocode)

1. transcribe(caller_audio) → transcript
2. extract_entities(transcript) → symptoms, keywords
3. urgency = triage_llm(transcript, symptoms, caller.history)
4. if urgency ≥ 80:
       dispatch = schedule_ambulance(location, high_priority)
       advise(caller, "Ambulance dispatched, ETA " + dispatch.eta)
   elif urgency ≥ 50:
       nearest_uc = find_urgent_care(location)
       advise(caller, "Drive to " + nearest_uc.name + " (~" + nearest_uc.drive_time + " min)")
   else:
       advise(caller, "Monitor symptoms, call back if worsening")
5. log_everything()

5. Agent Classification

Chosen Architecture: Hybrid (Deliberative + Reactive)

    Uses memory to accumulate caller history & symptom evolution across the call.
    Maintains a rolling plan (dispatch vs. advice) but can override immediately if new critical cues appear (e.g., “patient is unconscious now” triggers reactive dispatch).
    Justification: Purely reactive would miss multi-turn context; purely deliberative would introduce latency unacceptable for cardiac arrest calls.

6. Comparison to Reactive-Only Agent

| Aspect              | Hybrid (current)                                           | Pure Reactive                                    |
| ------------------- | ---------------------------------------------------------- | ------------------------------------------------ |
| **Memory**          | Keeps state across turns, enabling follow-up questions     | Stateless; each turn treated as new              |
| **Planning**        | Builds & updates a dispatch plan; can backtrack            | No plan; immediate pattern → action              |
| **Tool Invocation** | Batch queries (symptoms → triage → GIS) to reduce API cost | One tool per trigger; more API calls             |
| **Speed**           | Slight latency for deliberation                            | Fastest single-turn decisions                    |
| **Reliability**     | Cross-checks multiple models before dispatch               | Single point of failure if trigger rule is wrong |
| **Intelligence**    | Handles complex, multi-symptom cases                       | Fails on edge cases needing context              |


7. Reflection Answers

 What fails if state is not maintained?
    Without state, the agent cannot remember that chest pain evolved into unconsciousness two turns later, leading to under-triage and potentially fatal delays.
    Why are external tools essential?
    High-accuracy triage and up-to-date ambulance availability cannot be embedded in a single model; external APIs provide real-world data (traffic, hospital capacity) that the agent itself cannot simulate.

8. (Optional) GitHub Link

github.com/yourhandle/911-dispatch-agent-design